# <center> Introduction to Spark In-memory Computing via Python PySpark </center>

- Spark is an implementation of the MapReduce programming paradigm that operates on in-memory data and allows data reuses across multiple computations.
- Performance of Spark is significantly better than its predecessor, Hadoop MapReduce. 
- Spark's primary data abstraction is called a Resilient Distributed Dataset (RDD):
    - Read-only, partitioned collection of records
    - Created (aka written) through deterministic operations on data:
        - Loading from stable storage
        - Transforming from other RDDs
        - Generating through coarse-grained operations such as map, join, filter ...
    - Do not need to be materialized at all times and are recoverable via **data lineage**

<img src="figures/spark2_arch.png" width="600"/>

## 1. Getting Started

Spark stores data in memory. This memory space is represented by the variable **sc** (SparkContext). 

In [ ]:
!bash launch_spark_cluster.sh

In [ ]:
import sys
import os
import pyspark

env_spark_home=os.path.join(os.environ['HOME'],"software","spark-2.4.5-bin-hadoop2.7")
env_spark_conf_dir=os.path.join(env_spark_home,"conf")
env_pyspark_python=os.path.join("/software","anaconda3","5.1.0","bin","python")

os.environ['SPARK_HOME'] = env_spark_home
os.environ['SPARK_CONF_DIR'] = env_spark_conf_dir
os.environ['PYSPARK_PYTHON'] = env_pyspark_python

fp = open(os.path.join(env_spark_conf_dir,"master"))
node_list = fp.readlines()

import pyspark
conf = pyspark.SparkConf()
conf.setMaster("spark://" + node_list[0].strip() + ":7077")
conf.setAppName('big-data-workshop')
conf.set("spark.driver.memory","5g")
conf.set("spark.executor.instances", "2")
conf.set("spark.executor.memory","5g")
conf.set("spark.executor.cores","5")

sc = pyspark.SparkContext(conf=conf)

print(sc)

In [ ]:
textFile = sc.textFile("gutenberg-shakespeare.txt")

In [ ]:
print (textFile)

## 2. What does Spark do with my data?

**Storage Level:**
- Does RDD use disk?
- Does RDD use memory?
- Does RDD use off-heap memory?
- Should an RDD be serialized (while persisting)?
- How many replicas (default: 1) to use (can only be less than 40)?

In [ ]:
textFile.getStorageLevel()

In [ ]:
textFile.getNumPartitions()

In [ ]:
textFile.cache()

In [ ]:
textFile.getStorageLevel()

- By default, each transformed RDD may be recomputed each time you run an action on it.
- It is also possible to *persist* RDD in memory using *persist()* or *cache()*
    - *persist()* allows you to specify level of storage for RDD
    - *cache()* only persists RDD in memory
    - To retire RDD from memory, *unpersist()* is called

## 3. WordCount

Data operations in Spark are categorized into two groups, *transformation* and *action*. 
- A *transformation* creates new dataset from existing data. Examples of *transformation* include map, filter, reduceByKey, and sort. 
- An *action* returns a value to the driver program (aka memory space of this notebook) after running a computation on the data set. Examples of *action* include count, collect, reduce, and save. 

"All transformations in Spark are lazy, in that they do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result to be returned to the driver program." -- Spark Documentation

#### RDD Operations in Spark

**NOTE: ** See the Spark Programming Guide (for Python) for more detail on the following operations.

https://spark.apache.org/docs/2.2.0/rdd-programming-guide.html

**Transformations: **

- *map*(f: T -> U) : RDD[T] -> RDD[U]
- *filter*(f: T -> Bool) : RDD[T] -> RDD[T]
- *flatMap*(f: T -> Seq[U]) : RDD[T] -> RDD[U]
- *sample*(*fraction*: Float) : RDD[T] -> RDD[T] (deterministic sampling)
- *groupByKey*() : RDD[(K,V)] -> RDD[(K, Seq[V])]
- *reduceByKey*(f: (V,V) -> V) : RDD[(K,V)] -> RDD[(K,V)]
- *union*() : (RDD[T], RDD[T]) -> RDD[T]
- *join*() : (RDD[(K,V)], RDD[(K,W)]) -> RDD[(K,(V,W))]
- *cogroup*() : (RDD[(K,V)], RDD[(K,W)] -> RDD[(K, (Seq[V],Seq[W]))]
- *crossProduct*() : (RDD[T], RDD[U]) -> RDD[(T,U)]
- *mapValues*(f: V -> W) : RDD[(K,V)] -> RDD[(K,W)] (preserves partitioning)
- *sort*(c: Comparator[K]) :  RDD[(K,V)] -> RDD[(K,V)]
- *partitionBy*(p: Partitioner[K]) : RDD[(K,V)] -> RDD[(K,V)]

**Actions:**

- *count*() : RDD[T] -> Long
- *collect*() : RDD[T] -> Seq[T]
- *reduce*(f: (T,T) -> T) : RDD[T] -> T
- *lookup*(k : K) : RDD[(K,V)] -> Seq[V] (on hash/range partitionied RDDs)
- *save*(path: String) : Outputs RDD to a storage system 

In [ ]:
textFile = sc.textFile("gutenberg-shakespeare.txt")

In [ ]:
textFile

In [ ]:
%%time
textFile.count()

In [ ]:
wordcount = textFile.flatMap(lambda line: line.split(" ")) \
            .map(lambda word: (word, 1)) \
            .reduceByKey(lambda a, b: a + b)

In [ ]:
wordcount

In [ ]:
!rm -r ~/intro-to-spark
!mkdir ~/intro-to-spark

In [ ]:
wordcount.saveAsTextFile(os.path.join(os.environ['HOME'],"intro-to-spark/output-wordcount-01"))

In [ ]:
!ls -l ~/intro-to-spark/output-wordcount-01

In [ ]:
!cat ~/intro-to-spark/output-wordcount-01/part-00000 \
    2>/dev/null | head -n 20

**Step-by-step actions:**

In [ ]:
!cat gutenberg-shakespeare.txt \
    2>/dev/null | head -n 100

In [ ]:
wordcount_step_01 = textFile.flatMap(lambda line: line.split(" "))

In [ ]:
wordcount_step_01

In [ ]:
wordcount_step_01.take(20)

In [ ]:
wordcount_step_02 = wordcount_step_01.map(lambda word: (word, 1))

In [ ]:
wordcount_step_02.take(20)

In [ ]:
wordcount_step_03 = wordcount_step_02.reduceByKey(lambda a, b: a + b)

In [ ]:
wordcount_step_03.take(20)

### Challenge

- Augment the mapping process of WordCount with a function to filter out punctuations and capitalization from the unique words
  - Hint: The string module is helpful for removing punctuation.
  - Make sure your solution supports Python version 3.

In [ ]:
import string

In [ ]:
# PUT YOUR SOLUTION HERE

To stop the Spark job, call `sc.stop()`

In [ ]:
sc.stop()

In [ ]:
!bash stop_spark_cluster.sh